In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from path import Path
import pickle
from prettytable import PrettyTable
from scipy.signal import argrelextrema, find_peaks
plt.rcParams["font.family"] = "serif"
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14) 

In [ ]:
# First run the test.py to create the results pickle file for analysis
architectures = ['cnn', 'fc', 'vit', 'rvit', 'rcnn']
datasets = ['chua', 'img2force', 'mixed']
occlude_param = [None, "force_sensor"]
data = ['rgb']
save_dir = Path('results')

In [ ]:
# Load the data file or files
t_rmse = PrettyTable()
t_dtw = PrettyTable()
t_rmse.title = "Occluded parameters RMSE (N)"
t_rmse.field_names = ["Models"] + ["{}".format(op if op is not None else "none") for op in occlude_param]
results = {}
for s in datasets:
    for d in data:
        for arch in architectures:
            test_rmse_v, test_rmse_s = [], []
            for op in occlude_param:
                if op is not None:
                    file_s = save_dir/"{}/{}/{}/{}_state_random.pkl".format(s, d, op, arch)
                else:
                    file_s = save_dir/"{}/{}/{}_state_random.pkl".format(s, d, arch)
                with open(file_s, 'rb') as fileobject:
                    results_s = pickle.load(fileobject)       
                results["{}_{}_{}_{}_state".format(s, d, arch, "none" if op is None else op)] = results_s
                test_rmse_s.append((results_s['test_rmse'].mean() - results_s['test_rmse'].min()) / (results_s['test_rmse'].max() - results_s['test_rmse'].min()))

            t_rmse.add_row(["{}_{}_{}_state".format(s, d, arch)] + ["{:.2f}".format(a) for a in test_rmse_s])

print(t_rmse)

In [ ]:
# Cell to plot the force estimation for the unseen testing set
my_dpi = 100
labels = ['X', 'Y', 'Z']
ranges = {"chua": [[30*10, 30*13], [30*10, 30*15], [30*84, 30*87]],
          "img2force": [[30*10, 30*13], [30*10, 30*15], [30*10, 30*15]],
          "mixed": [[30*10, 30*13], [30*10, 30*15], [30*84, 30*87]]}

save_fig_root = Path('figures')  
for s in datasets:
    x = (1/30)*np.linspace(0, len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']), len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']))
    x_rnn = (1/30)*np.linspace(0, len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']), len(results['{}_rgb_rcnn_force_sensor_state'.format(s)]['test_pred']))
    range_s = ranges[s]
    for op in occlude_param:
        if op is None:
            continue
        for d in data:
            fig, axs = plt.subplots(ncols=1, nrows=3, sharex=False, sharey=False, figsize=(1920/my_dpi, 1080/my_dpi), dpi=my_dpi)
            fig.suptitle("{} force estimation with occluded {}".format(s, op), fontsize=24, fontweight='bold')
            for j, ax in enumerate(axs):
                r = range_s[j]
                ax.plot(x[r[0]:r[1]], results["{}_{}_cnn_{}_state".format(s, d, op)]['test_gt'][r[0]:r[1], j], 'b', linewidth=3.0)
                ax.plot(x[r[0]:r[1]], results["{}_{}_cnn_{}_state".format(s, d, op)]['test_pred'][r[0]:r[1], j], 'g', linewidth=3.0)
                ax.plot(x[r[0]:r[1]], results['{}_{}_vit_{}_state'.format(s, d, op)]['test_pred'][r[0]:r[1], j], 'y', linewidth=3.0)
                ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rcnn_{}_state'.format(s, d, op)]['test_pred'][r[0]:r[1], j], 'r', linewidth=3.0)
                ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rvit_{}_state'.format(s, d, op)]['test_pred'][r[0]:r[1], j], 'k', linewidth=3.0)
                ax.plot(x[r[0]:r[1]], results['{}_{}_fc_{}_state'.format(s, d, op)]['test_pred'][r[0]:r[1], j], 'm', linewidth=3.0)
                # ax.legend(["GT", "CNN", "CNN-BAM", "RNN", "RNN-BAM", 'FC'], fontsize=14)
                ax.legend(["GT", "CNN", "VIT", "R-CNN", "R-VIT", "FC"], fontsize=14, loc=5)
                ax.set_ylabel("Force {} (N)".format(labels[j]), fontsize=16, fontweight='bold')
                if j == 2:
                    ax.set_xlabel("Time (s)", fontsize=16, fontweight='bold')

            fig.align_labels()
            save_fig_path = save_fig_root/"{}/{}".format(s, op)
            save_fig_path.makedirs_p()
            fig.savefig(save_fig_path/"random.png", dpi=my_dpi)
            plt.show()

In [ ]:
my_dpi = 100
ranges = {"chua": [30*10, 30*13],
          "img2force": [30*10, 30*13],
          "mixed": [30*84, 30*87]}

for s in datasets:
    x = (1/30)*np.linspace(0, len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']), len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']))
    x_rnn = (1/30)*np.linspace(0, len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']), len(results['{}_rgb_rcnn_force_sensor_state'.format(s)]['test_pred']))
    range_s = ranges[s]
    for op in occlude_param:
        if op is None:
            continue
        for d in data:
            r = range_s
            fig, ax = plt.subplots(ncols=1, nrows=1, sharex=False, sharey=False, figsize=(1920/my_dpi, 1080/my_dpi), dpi=my_dpi)
            fig.suptitle("{} evolution of RMSE for {} occlusion".format(s, op), fontsize=24, fontweight='bold')
            # ax.plot(x, results['{}_{}_cnn_state_{}'.format(s, d, f)]['test_gt'][r[0]:r[1], j], 'b', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_cnn_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'g', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_vit_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'y', linewidth=3.0)
            ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rcnn_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'r', linewidth=3.0)
            ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rvit_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'k', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_fc_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'm', linewidth=3.0)
            # ax.legend(["GT", "CNN", "CNN-BAM", "RNN", "RNN-BAM", 'FC'], fontsize=14)
            ax.legend(["CNN", "VIT", "R-CNN", "R-VIT", "FC"], fontsize=14, loc=1)
            ax.set_ylabel("RMSE (N)", fontsize=16, fontweight='bold')
            ax.set_xlabel("Time (s)", fontsize=16, fontweight='bold')
        
        save_fig_path = save_fig_root/"{}/{}".format(s, op)
        save_fig_path.makedirs_p()
        fig.align_labels()
        fig.savefig(save_fig_path/'metrics.png', dpi=my_dpi)
        plt.show()

In [ ]:
fig = plt.figure(figsize=(1920/my_dpi, 1080/my_dpi))
max_peaks_x, _ = find_peaks(results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], distance=150)
max_peaks_y, _ = find_peaks(results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], distance=150)
max_peaks_z, _ = find_peaks(results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], distance=150)

min_peaks_x, _ = find_peaks(-results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], distance=150)
min_peaks_y, _ = find_peaks(-results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], distance=150)
min_peaks_z, _ = find_peaks(-results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], distance=150)

plt.subplot(3, 1, 1)
plt.plot(x, results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], linewidth=2.0)
plt.plot(x[max_peaks_x], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_x, 0], '*', markersize=10.0)
plt.plot(x[min_peaks_x], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_x, 0], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force X (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

plt.subplot(3, 1, 2)
plt.plot(x, results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], linewidth=2.0)
plt.plot(x[max_peaks_y], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_y, 1], '*', markersize=10.0)
plt.plot(x[min_peaks_y], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_y, 1], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force Y (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

plt.subplot(3, 1, 3)
plt.plot(x, results["chua_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], linewidth=2.0)
plt.plot(x[max_peaks_z], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_z, 2], '*', markersize=10.0)
plt.plot(x[min_peaks_z], results["chua_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_z, 2], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force Z (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

fig.align_labels()

fig.savefig("figures/max_min_force.png", dpi=my_dpi)

In [ ]:
fig = plt.figure()
for s in datasets:
    for d in data:
        for op in occlude_param:
            if op is None: continue
            peaks_x, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["test_gt"][:, 0], distance=150)
            peaks_y, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["test_gt"][:, 1], distance=150)
            peaks_z, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["test_gt"][:, 2], distance=150)
            for arch in architectures:
                rmse_x = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["test_rmse"][peaks_x].mean()
                rmse_y = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["test_rmse"][peaks_y].mean()
                rmse_z = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["test_rmse"][peaks_z].mean()
                langs = ["X", "Y", "Z"]
                rmse = np.mean([rmse_x, rmse_y, rmse_z])
                print("Value for {}_{}_{}_{}: {} N".format(s, d, arch, op, rmse))
                # plt.bar(langs, rmse)
                # plt.pause(3.0)
                
